In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
page = requests.get('https://yts.mx/browse-movies/0/all/action/5/latest/0/all')
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
links = []

results = soup.find('section')

# Ladies and gentlemen, listen up. Just so you understand, you are allowed to be tough. And I am not allowed to give up!
for link in results.find_all('div', class_ = "browse-movie-bottom"):
    links.append(link.find('a')['href'].strip())
    
links[0]

'https://yts.mx/movies/trap-of-lust-1973'

In [4]:
columns = ['Name', 'Genre', 'Likes', 'IMDB', 'Year', 'Language', 'critics', 'audience']
df = pd.DataFrame(columns = columns)
df

,Name,Genre,Likes,IMDB,Year,Language,critics,audience


In [6]:

for link in links:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    results = soup.find('div', attrs = {'id': 'movie-info'})
    
    name = results.find('h1').get_text().strip()
    genre = ', '.join(results.find_all('h2')[1].get_text().strip().split(' / '))
    try:
        likes = results.find('span', attrs = {'id': 'movie-likes'}).get_text().strip()
    except:
        likes = ''
    imdb = float(results.find('span', attrs = {'itemprop': "ratingValue"}).get_text().strip())
    year_lan = results.find_all('h2')[0].get_text().strip().split()
    if len(year_lan) == 1:
        year = year_lan[0]
        language = 'English'
    elif len(year_lan) ==2:
        year = year_lan[0]
        language = year_lan[1].replace('[', '').replace(']', '').title()
    try:
        critics = results.find('div', class_ = 'bottom-info').find_all('div', class_ = 'rating-row')[1].find('span').get_text()
        if '%' not in critics:
            critics = ''
    except:
        critics = ''
    try:
        audience = results.find('div', class_ = 'bottom-info').find_all('div', class_ = 'rating-row')[2].find('span').get_text()
        if '%' not in audience:
            audience = ''
    except:
        audience = ''
    
    #print(name, critics, audience)
    
    headers = ['Name', 'Genre', 'Likes', 'IMDB', 'Year', 'Language', 'critics', 'audience']
    items = [name, genre, likes, imdb, year, language, critics, audience]
    
    data = {i: [j] for i, j in zip(headers, items)}
    
    temp_df = pd.DataFrame(data)
    df = pd.concat([df, temp_df], ignore_index = True)

df.head(10)

,Name,Genre,Likes,IMDB,Year,Language,critics,audience
0,Trap of Lust,"Action, Crime",1,5.8,1973,Japanese,,
1,Babes in Toyland,"Action, Adventure, Family, Fantasy, Musical, R...",3,5.4,1986,English,50%,
2,Time,"Action, Documentary",7,7.2,2020,English,98%,48%
3,Devil's Playground,"Action, Horror",1,5.1,2010,English,21%,
4,To Your Last Death,"Action, Animation, Fantasy, Horror",31,7.0,2019,English,69%,83%
5,Ready or Not,"Action, Comedy, Horror, Mystery, Thriller",285,6.8,2019,English,88%,78%
6,Into the Sun,"Action, Comedy, War",3,5.3,1991,English,,
7,Crazed Fruit,Action,1,6.8,1981,Japanese,,
8,Authentic True Account: Osaka Shock Tactics,"Action, Crime",0,7.5,1976,Japanese,,
9,Beautiful Thing,"Action, Comedy, Drama, Romance",19,7.5,1996,English,91%,93%


In [7]:
for i in range(1, 101):
    url = 'https://yts.mx/browse-movies/0/all/action/5/latest/0/all?page={}'.format(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    links = []

    results = soup.find('section')
    # Ladies and gentlemen, listen up. Just so you understand, you are allowed to be tough. And I am not allowed to give up!
    for link in results.find_all('div', class_ = "browse-movie-bottom"):
        links.append(link.find('a')['href'].strip())
    
    
    for link in links:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')

        results = soup.find('div', attrs = {'id': 'movie-info'})

        try:
            name = results.find('h1').get_text().strip()
        except:
            name = ''
        try:
            genre = ', '.join(results.find_all('h2')[1].get_text().strip().split(' / '))
        except:
            genre = ''
        try:
            likes = results.find('span', attrs = {'id': 'movie-likes'}).get_text().strip()
        except:
            likes = ''
        try:
            imdb = results.find('span', attrs = {'itemprop': "ratingValue"}).get_text().strip()
        except:
            imbd = ''
        
        try:
            year_lan = results.find_all('h2')[0].get_text().strip().split()
            if len(year_lan) == 1:
                year = year_lan[0]
                language = 'English'
            elif len(year_lan) ==2:
                year = year_lan[0]
                language = year_lan[1].replace('[', '').replace(']', '').title()
        except:
            year_lan = ''
        try:
            critics = results.find('div', class_ = 'bottom-info').find_all('div', class_ = 'rating-row')[1].find('span').get_text()
            if '%' not in critics:
                critics = ''
        except:
            critics = ''
        try:
            audience = results.find('div', class_ = 'bottom-info').find_all('div', class_ = 'rating-row')[2].find('span').get_text()
            if '%' not in audience:
                audience = ''
        except:
            audience = ''

        headers = ['Name', 'Genre', 'Likes', 'IMDB', 'Year', 'Language', 'critics', 'audience']
        items = [name, genre, likes, imdb, year, language, critics, audience]

        data = {i: [j] for i, j in zip(headers, items)}

        temp_df = pd.DataFrame(data)
        df = pd.concat([df, temp_df], ignore_index = True)

df
    

,Name,Genre,Likes,IMDB,Year,Language,critics,audience
0,Trap of Lust,"Action, Crime",1,5.8,1973,Japanese,,
1,Babes in Toyland,"Action, Adventure, Family, Fantasy, Musical, R...",3,5.4,1986,English,50%,
2,Time,"Action, Documentary",7,7.2,2020,English,98%,48%
3,Devil's Playground,"Action, Horror",1,5.1,2010,English,21%,
4,To Your Last Death,"Action, Animation, Fantasy, Horror",31,7,2019,English,69%,83%
...,...,...,...,...,...,...,...,...
2015,End of Summer,"Action, Drama, Romance",2,5.2,1997,English,89%,
2016,Sno Babies,"Action, Drama",4,5.7,2020,English,25%,99%
2017,A Resistance,"Action, Drama",2,6.4,2019,Korean,,
2018,Max and Me,"Action, Drama, Romance",6,6.3,2020,English,,
